In [1]:
import GridMLM_tokenizers
from GridMLM_tokenizers import CSGridMLMTokenizer
from models_graph import make_graph_from_input_ids, remove_consecutive_duplicates, remove_out_of_dict_ids, compute_edge_features
from data_utils import CSGridMLMDataset
import torch
from tqdm import tqdm

/home/maximos/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = CSGridMLMTokenizer(fixed_length=256)
chord_features = GridMLM_tokenizers.CHORD_FEATURES
chord_id_features = {tokenizer.vocab[k]: v for k, v in chord_features.items()}
print(chord_features)

{'C:maj': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], 'C#:maj': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 'D:maj': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0], 'D#:maj': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], 'E:maj': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], 'F:maj': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], 'F#:maj': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 'G:maj': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], 'G#:maj': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], 'A:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 'A#:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 'B:maj': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [3]:
print(chord_features['C:maj'])
print(chord_features['D:maj'])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]


In [4]:
print(chord_id_features[7])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]


In [5]:
train_dir = '/mnt/ssd2/maximos/data/hooktheory_midi_hr/CA_train'
val_dir = '/mnt/ssd2/maximos/data/hooktheory_midi_hr/CA_test'
# train_dir = '/media/maindisk/data/hooktheory_midi_hr/CA_train'
# val_dir = '/media/maindisk/data/hooktheory_midi_hr/CA_test'

In [6]:
# train_dataset = GuidedGridMLMDataset(train_dir, tokenizer, 512, frontloading=True)
val_dataset = CSGridMLMDataset(val_dir, tokenizer, frontloading=True, name_suffix='Q4_L80_bar_PC')

Loading data file.


In [7]:
print(val_dataset[0])

{'harmony_ids': [6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 6, 269, 269, 269, 269, 6, 152, 152, 66, 66, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'pianoroll': array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'time_signature': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'h_density_complexity': [0, 1, 0, 0, 1, 0, 0, 0]}


In [8]:
chord_id_duplicates_sequence = val_dataset[6]['harmony_ids']

In [9]:
print(chord_id_duplicates_sequence)

[6, 7, 7, 7, 210, 6, 210, 210, 210, 210, 6, 269, 269, 269, 13, 6, 13, 13, 13, 13, 6, 152, 152, 152, 152, 6, 152, 152, 152, 152, 6, 7, 7, 7, 7, 6, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
chord_ids_sequence = remove_consecutive_duplicates( chord_id_duplicates_sequence )
print(chord_ids_sequence)
chord_ids_sequence = remove_out_of_dict_ids( chord_ids_sequence, chord_id_features )
print(chord_ids_sequence)
unique_chord_ids_sequence = list(set(chord_ids_sequence))
print(unique_chord_ids_sequence)
for c in unique_chord_ids_sequence:
    print(c, tokenizer.ids_to_tokens[c], chord_id_features[c])

[6, 7, 210, 6, 210, 6, 269, 13, 6, 13, 6, 152, 6, 152, 6, 7, 6, 7, 1]
[7, 210, 210, 269, 13, 13, 152, 152, 7, 7]
[7, 13, 269, 210, 152]
7 C:maj [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
13 C:7 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
269 A:min [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
210 G:maj [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
152 F:maj [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0]


In [11]:
src_id = unique_chord_ids_sequence[0]
dst_id = unique_chord_ids_sequence[1]

src_feat = torch.LongTensor(chord_id_features[src_id])
dst_feat = torch.LongTensor(chord_id_features[dst_id])

feats = compute_edge_features(src_feat, dst_feat)
print(src_id, tokenizer.ids_to_tokens[src_id], src_feat)
print(dst_id, tokenizer.ids_to_tokens[dst_id], dst_feat)
print('Features: ', feats)

7 C:maj tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])
13 C:7 tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
Features:  tensor([0.0000, 0.7500])


In [12]:
# chord_id_duplicates_sequence = [10,11,12,10]
g = make_graph_from_input_ids(chord_id_duplicates_sequence, chord_id_features, use_probabilities=True)

In [44]:
print(g)

Data(x=[4, 24], edge_index=[2, 7], edge_attr=[7, 2], node_ids=[4], edge_weight=[7])


In [13]:
# make graph dataset

graph_val_dataset = []

for d in tqdm(val_dataset):
    chord_id_duplicates_sequence = d['harmony_ids']
    g = make_graph_from_input_ids(
            chord_id_duplicates_sequence,
            chord_id_features,
            use_probabilities=True
        )
    if g is not None:
        graph_val_dataset.append( g )
    else:
        print('Short sequence: ', chord_id_duplicates_sequence)

100%|██████████| 758/758 [00:00<00:00, 5915.02it/s]

Short sequence:  [6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 4, 4, 4, 4, 6, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Short sequence:  [6, 4, 4, 4, 4, 6, 4, 4, 4, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
print(chord_id_duplicates_sequence)
# print(g)
print(g.x)
print(g.node_ids)
print(g.edge_index)
print(g.edge_attr)
print(g.edge_weight)

[6, 269, 269, 269, 269, 6, 123, 123, 123, 123, 6, 123, 123, 123, 123, 6, 269, 269, 269, 269, 6, 269, 7, 7, 7, 6, 66, 66, 66, 66, 6, 123, 123, 123, 123, 6, 269, 269, 269, 269, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0.]])
tensor([ 66, 123, 269,   7])
tensor([[2, 1, 1, 2, 2, 3, 0],
        [1, 1, 2, 2, 3, 0, 1]])
tensor([[0.1667, 0.2000],
        [0.0000, 1.0000],
        [0.1667, 0.2000],
        [0.0000, 1.0000],
        [0.5000, 0.5000],
        [0.3333, 0.000

In [15]:
from models_graph import HarmonicGraphEncoder, BilinearDecoder, HarmonicGAE, edge_index_to_dense

In [16]:
hidden_dim = 64

encoder = HarmonicGraphEncoder(hidden_dim=hidden_dim)
decoder = BilinearDecoder(hidden_dim=hidden_dim)
gae = HarmonicGAE(hidden_dim=hidden_dim)


In [17]:
z, z_pooled = encoder(g)

In [18]:
print(z.shape)
print(z_pooled.shape)

torch.Size([4, 64])
torch.Size([1, 64])


In [19]:
x = decoder(z)

In [20]:
print(x)

tensor([[0.2739, 0.1973, 0.2520, 0.2768],
        [0.5054, 0.0107, 0.0484, 0.4356],
        [0.4471, 0.0410, 0.1042, 0.4077],
        [0.2995, 0.1697, 0.2327, 0.2980]], grad_fn=<SoftmaxBackward0>)


In [21]:
y, enc_pooled = gae( g )

In [22]:
print(y)
print(enc_pooled)

tensor([[0.3313, 0.1543, 0.2136, 0.3007],
        [0.1467, 0.4280, 0.2573, 0.1680],
        [0.1987, 0.3199, 0.2542, 0.2272],
        [0.3320, 0.1488, 0.2081, 0.3111]], grad_fn=<SoftmaxBackward0>)
tensor([[-0.3606, -0.7241,  0.5362, -0.7869,  0.0849,  0.3201,  1.1003, -1.7041,
          0.4257,  0.5154,  0.1330,  0.0591, -0.7173, -0.6782,  0.7893, -0.2299,
         -0.7237, -0.0855,  0.6955,  0.6677,  1.0698,  0.5029, -0.2820,  0.3652,
         -0.6681, -1.0766,  0.6376,  0.8644,  0.7708,  0.1223,  0.1113,  0.3640,
         -0.4276,  0.5092,  0.4926, -0.2603,  0.2774,  0.8284,  0.4534,  0.8539,
         -0.0446,  0.2949,  0.4361, -0.3755, -0.0754, -0.3661, -0.4415,  0.4981,
         -0.4401, -0.4578,  0.1745, -0.1551,  0.7120,  0.7527, -0.0854, -0.7268,
          1.1688,  0.0164, -0.7460,  0.3555,  1.5389,  1.0326, -1.3506,  0.8274]],
       grad_fn=<UnsqueezeBackward0>)


In [23]:
target = edge_index_to_dense(g)

loss_fn = torch.nn.KLDivLoss(reduction="batchmean")
loss = loss_fn(torch.log(y + 1e-9), target)

In [24]:
print(target)

tensor([[0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.3333, 0.6667, 0.0000],
        [0.0000, 0.3333, 0.3333, 0.3333],
        [1.0000, 0.0000, 0.0000, 0.0000]])


In [25]:
print(loss.item())

0.9385910034179688


In [43]:
print(len(graph_val_dataset))
g0 = graph_val_dataset[3]
print(g0)

756
Data(x=[4, 24], edge_index=[2, 3], edge_attr=[3, 2], node_ids=[4], edge_weight=[3])


In [47]:
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool

loader = DataLoader(graph_val_dataset, batch_size=32, shuffle=False)

In [48]:
print(len(loader))

24


In [49]:
b = next(iter(loader))

In [50]:
print(b)
print(b.batch)

DataBatch(x=[151, 24], edge_index=[2, 197], edge_attr=[197, 2], node_ids=[151], edge_weight=[197], batch=[151], ptr=[33])
tensor([ 0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,  4,  4,  4,  5,
         5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11,
        11, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15,
        15, 16, 16, 16, 16, 16, 17, 17, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 23,
        23, 23, 23, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27,
        27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30,
        30, 31, 31, 31, 31, 31, 31])


In [57]:
# for training
loss_fn = torch.nn.KLDivLoss(reduction="batchmean")

for batch in loader:
    y, _ = gae(batch)
    graphs = batch.to_data_list() # get list of each graph
    batch_loss = None
    for g in graphs:
        target = edge_index_to_dense(g)
        if batch_loss is None:
            batch_loss = loss_fn(torch.log(y + 1e-9), target)
        else:
            batch_loss += loss_fn(torch.log(y + 1e-9), target)

RuntimeError: The size of tensor a (3) must match the size of tensor b (151) at non-singleton dimension 1

In [ ]:
# for collecting embeddings
embeddings = []

for batch in loader:
    z, _ = encoder(batch)
    graph_embeddings = global_mean_pool(z, batch.batch)
    embeddings.append(graph_embeddings.detach())

embeddings = torch.cat(embeddings, dim=0)

In [52]:
print(embeddings.shape)

torch.Size([756, 64])
